# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon, make_map
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from regions import boolean_mask, global_ocean, gl_ocean_rect, SST_index_bounds, boolean_mask
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA, dll_dims_names
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, xr_lintrend, xr_quadtrend

In [ ]:
runs = ['ctrl', 'rcp', 'lpd', 'lpi', 'had']
domains = ['ocn', 'ocn', 'ocn_low', 'ocn_low', 'ocn_had']
map_domains = ['ocn_T', 'ocn_T', 'ocn_T', 'ocn_T', 'ocn_had']

In [ ]:
MASK_ocn = boolean_mask(domain='ocn', mask_nr=0, rounded=True)
MASK_low = boolean_mask(domain='ocn_low', mask_nr=0, rounded=True)
MASK_had = boolean_mask(domain='ocn_had', mask_nr=0, rounded=True)
AREA_ocn = xr_AREA(domain='ocn'    )
AREA_low = xr_AREA(domain='ocn_low')
AREA_had = xr_AREA(domain='ocn_had')
masks = [MASK_ocn, MASK_ocn, MASK_low, MASK_low, MASK_had]

# 1. SST: Mean, Trends, Biases, Time Series

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc', decode_times=False)
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' , decode_times=False)
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' , decode_times=False)
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' , decode_times=False)
SST_yrly_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_had.nc' , decode_times=False)

When the yrly SST data of the CTRL run is created, some monthly file around year 200 are not saved on January 31, but rather Jan 15 or 16, so the time coordinate is not equally spaced. I fixed this manually by loading the dataset and performing `SST_yrly_ctrl = SST_yrly_ctrl.assign_coords(time=np.arange(100,300)*365+31)` and then saving this file.

In [ ]:
# SST_yrly_ctrl = SST_yrly_ctrl.assign_coords(time=np.arange(100,300)*365+31)
# SST_yrly_ctrl.to_netcdf(f'{path_samoc}/SST/SST_yrly_ctrl2.nc')

## 1.1 Mean

In [ ]:
%%time
SST_mean_ctrl = SST_yrly_ctrl[-50:].where(MASK_ocn).mean(dim='time')
SST_mean_rcp  = SST_yrly_rcp [-50:].where(MASK_ocn).mean(dim='time')
SST_mean_lpd  = SST_yrly_lpd [-50:].where(MASK_low).mean(dim='time')
SST_mean_lpi  = SST_yrly_lpi [-50:].where(MASK_low).mean(dim='time')
SST_mean_had  = SST_yrly_had [-50:].where(MASK_had).mean(dim='time')

In [ ]:
fn  = f'{path_results}/SST/SST_mean_had'
cm  = discrete_cmap(17, cmocean.cm.thermal)
txt1 = 'HadISST'
txt2 = '1969-2018\nmean'
f, ax = make_map(xa=SST_mean_had, domain='ocn_had', proj='rob', cmap=cm, minv=-2, maxv=32,
                 label='SST [$^\circ$C]', filename=fn, text1=txt1, text2=txt2, clon=200)

## 1.2 Timeseries of global mean SST

In [ ]:
# calculating global mean
# SST_global_mean_ctrl = (SST_yrly_ctrl*AREA_ocn).where(MASK_ocn).sum(axis=(1,2))/(AREA_ocn.where(MASK_ocn).sum())
# SST_global_mean_rcp  = (SST_yrly_rcp *AREA_ocn).where(MASK_ocn).sum(axis=(1,2))/(AREA_ocn.where(MASK_ocn).sum())
# SST_global_mean_lpd  = (SST_yrly_lpd *AREA_low).where(MASK_low).sum(axis=(1,2))/(AREA_low.where(MASK_low).sum())
# SST_global_mean_lpi  = (SST_yrly_lpi *AREA_low).where(MASK_low).sum(axis=(1,2))/(AREA_low.where(MASK_low).sum())
# SST_global_mean_had  = (SST_yrly_had *AREA_had).where(MASK_had).sum(axis=(1,2))/(AREA_had.where(MASK_had).sum())

In [ ]:
# writing global mean to netcdf
# SST_global_mean_ctrl.to_netcdf(f'{path_samoc}/SST/SST_global_mean_timeseries_ctrl.nc')
# SST_global_mean_rcp .to_netcdf(f'{path_samoc}/SST/SST_global_mean_timeseries_rcp.nc')
# SST_global_mean_lpd .to_netcdf(f'{path_samoc}/SST/SST_global_mean_timeseries_lpd.nc')
# SST_global_mean_lpi .to_netcdf(f'{path_samoc}/SST/SST_global_mean_timeseries_lpi.nc')
# SST_global_mean_had .to_netcdf(f'{path_samoc}/SST/SST_global_mean_timeseries_had.nc')

In [ ]:
# loading GMSST from previously saved netcdf
SST_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_global_mean_timeseries_ctrl.nc')
SST_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_global_mean_timeseries_rcp.nc')
SST_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_global_mean_timeseries_lpd.nc')
SST_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_global_mean_timeseries_lpi.nc')
SST_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_global_mean_timeseries_had.nc')

In [ ]:
f, ax = plt.subplots(1,1,figsize=(12,5), sharex=True)
ax.tick_params(labelsize=14)
# ax.axhline(0,c='k', lw=.5)

time_had = np.arange(2350,2519)

ax.axhline(0, c='k', lw=.5)
ax.plot(SST_ctrl.time/365+1850, SST_ctrl-SST_ctrl.mean(), c='C0')
ax.plot(SST_rcp.time/365 + 200, (SST_rcp-SST_rcp .mean())/3, c='C1')
ax.plot(SST_lpd.time/365 +1350, SST_lpd -SST_lpd .mean(), c='C2')
ax.plot(SST_lpi.time/365 -1600, SST_lpi -SST_lpi .mean(), c='C3')
ax.plot(np.arange(2370,2519)  , SST_had -SST_had .mean(), c='C4')


ax.text(1950, .45, 'CTRL'         , fontsize=16, color='C0')
ax.text(2150, .45, 'RCP'          , fontsize=16, color='C1')
ax.text(2150, .38, r'$\times\frac{1}{3}$' , fontsize=20, color='C1')
ax.text(1500, .45, 'pres. day low', fontsize=16, color='C2')
ax.text(   0, .45, 'pre-ind. low' , fontsize=16, color='C3')
ax.text(2320, .45, 'HadISST'      , fontsize=16, color='C4')

# ax.legend(handles=[L1, L2, L3], loc=8, ncol=3, fontsize=14, frameon=False)
ax.set_xlabel('time [years]', fontsize=16)
ax.set_ylabel('GMSST [$^\circ$C]', fontsize=16)

ax.set_xticks(np.arange(0,2800,200))
ax.set_xlim((-50,2550))
f.align_ylabels()
f.tight_layout()
# plt.savefig(f'{path_results}/SST/SST_global_mean_timeseries')

## 1.2: Trend at each point

In [ ]:
%%time
# 2 min
SST_trend_ctrl, SST_trend_interc_ctrl = ocn_field_regression(SST_yrly_ctrl, run='ctrl')
SST_trend_rcp , SST_trend_interc_rcp  = ocn_field_regression(SST_yrly_rcp , run='rcp' )
SST_trend_lpd , SST_trend_interc_lpd  = ocn_field_regression(SST_yrly_lpd , run='lpd' )
SST_trend_lpi , SST_trend_interc_lpi  = ocn_field_regression(SST_yrly_lpi , run='lpi' )

In [ ]:
# example point
SST_yrly_lpd[:,100,100].plot()
(SST_trend_lpd[100,100]*SST_yrly_lpd.time + SST_trend_interc_lpd[100,100]).plot()

In [ ]:
cmap  = discrete_cmap(12, shifted_color_map(cmocean.cm.balance, start=1/3, midpoint=.5, stop=5/6, name='shrunk'))

xa    = SST_trend_ctrl*100*365
fn    = f'{path_results}/SST/SST_trend_ctrl'
label = '100-299 SST trend [K/century]'
txt1  = f'CTRL'
f, ax = make_map(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4,
                 label=label, filename=fn, text1=txt1)

xa    = SST_trend_rcp*100*365
fn    = f'{path_results}/SST/SST_trend_rcp'
label = '2000-2099 SST trend [K/century]'
txt1  = f'RCP'
f, ax = make_map(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4,
                 label=label, filename=fn, text1=txt1)

## 1.3 Mean Bias

In [ ]:
SSTs = [SST_yrly_ctrl, SST_yrly_rcp, SST_yrly_lpd, SST_yrly_lpi, SST_yrly_had]

In [ ]:
%%time
for i, SST_ac in enumerate(SSTs):
    #     if i!=2: continue
    run = runs[i]
    fn = f'{path_samoc}/SST/SST_autocorrelation_{run}.nc'
    fa = FieldAnalysis(SST_ac[-100:])
    xa = fa.make_autocorrelation_map(fn=fn)
    
    fn = f'{path_results}/SST/SST_autocorrelation_map_{run}'
    domain = map_domains[i]
    label = 'autocorrelation of SST'
    cmap = cmocean.cm.curl
    txt1 = f'{run.upper()}\ndetr.'
    txt2 = '100 years'
    make_map(xa=xa, domain=domain, proj='rob', cmap=cmap, minv=-1, maxv=1,
             label=label, filename=fn, text1=txt1, text2=txt2)

# Ocean basins and SST index regions

## Global mean vs. 60S-60N time series

In [ ]:
# SST_gm_ctrl      = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_ctrl.nc'     , decode_times=False)
# SST_gm_rcp       = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rcp.nc'      , decode_times=False)
SST_gm_ctrl = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_ctrl.nc' , decode_times=False)
SST_gm_rcp  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_rcp.nc'  , decode_times=False)
SST_gm_lpd  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_lpd.nc'       , decode_times=False)
SST_gm_lpi  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_lpi.nc'       , decode_times=False)

SST_xm_ctrl = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_rect_ctrl.nc', decode_times=False)
SST_xm_rcp  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_rect_rcp.nc' , decode_times=False)
SST_xm_lpd  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_lpd.nc'      , decode_times=False)
SST_xm_lpi  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_lpi.nc'      , decode_times=False)
SST_xm_had  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_had.nc'      , decode_times=False)

In [ ]:
# deseasonalize
SST_gm_ds_ctrl = deseasonalize(SST_gm_ctrl)
SST_gm_ds_rcp  = deseasonalize(SST_gm_rcp )
SST_gm_ds_lpd  = deseasonalize(SST_gm_lpd )
SST_gm_ds_lpi  = deseasonalize(SST_gm_lpi )
SST_xm_ds_ctrl = deseasonalize(SST_xm_ctrl)
SST_xm_ds_rcp  = deseasonalize(SST_xm_rcp )
SST_xm_ds_lpd  = deseasonalize(SST_xm_lpd )
SST_xm_ds_lpi  = deseasonalize(SST_xm_lpi )
SST_xm_ds_had  = deseasonalize(SST_xm_had )

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot([-1.5,2], [-1  ,2.5], c='k', lw=.5)
plt.plot([-1.5,2], [-1.5,2  ], c='k', lw=.5)
plt.plot([-1.5,2], [-0.5,3  ], c='k', lw=.5)
plt.plot([-1.5,2], [-2  ,1.5], c='k', lw=.5)
plt.xlim((-1,1.5))
plt.ylim((-1,1.5))
plt.scatter(SST_xm_ds_ctrl-SST_xm_ds_ctrl.mean(dim='time'), SST_gm_ds_ctrl-SST_gm_ds_ctrl.mean(dim='time')+.5, alpha=.1)
plt.scatter(SST_xm_ds_rcp -SST_xm_ds_rcp .mean(dim='time'), SST_gm_ds_rcp -SST_gm_ds_rcp .mean(dim='time')   , alpha=.1)
plt.scatter(SST_xm_ds_lpd -SST_xm_ds_lpd .mean(dim='time'), SST_gm_ds_lpd -SST_gm_ds_lpd .mean(dim='time')+1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi -SST_xm_ds_lpi .mean(dim='time'), SST_gm_ds_lpi -SST_gm_ds_lpi .mean(dim='time')-.5, alpha=.1)
plt.scatter(SST_xm_ds_had -SST_xm_ds_had .mean(dim='time'), [-.9]*len(SST_xm_ds_had), alpha=.1)
plt.ylabel('global mean SST' , fontsize=14)
plt.xlabel('60S-60N mean SST', fontsize=14)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot([-1.5,2], [-1  ,2.5], c='k', lw=.5)
plt.plot([-1.5,2], [-1.5,2  ], c='k', lw=.5)
plt.plot([-1.5,2], [-0.5,3  ], c='k', lw=.5)
plt.plot([-1.5,2], [-2  ,1.5], c='k', lw=.5)
plt.xlim((-.3,.3))
plt.ylim((-1.3,1.4))
plt.scatter(SST_xm_ds_ctrl-xr_quadtrend(SST_xm_ds_ctrl), SST_gm_ds_ctrl-xr_quadtrend(SST_gm_ds_ctrl)+.5, alpha=.1)
plt.scatter(SST_xm_ds_rcp -xr_quadtrend(SST_xm_ds_rcp ), SST_gm_ds_rcp -xr_quadtrend(SST_gm_ds_rcp )   , alpha=.1)
plt.scatter(SST_xm_ds_lpd -xr_quadtrend(SST_xm_ds_lpd ), SST_gm_ds_lpd -xr_quadtrend(SST_gm_ds_lpd )+1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi -xr_quadtrend(SST_xm_ds_lpi ), SST_gm_ds_lpi -xr_quadtrend(SST_gm_ds_lpi )-.5, alpha=.1)
plt.scatter(SST_xm_ds_had -xr_quadtrend(SST_xm_ds_had ), [-1]*len(SST_xm_ds_had), alpha=.1)

plt.scatter(SST_xm_ds_lpd[-200*12:] -xr_quadtrend(SST_xm_ds_lpd[-200*12:] ),
            SST_gm_ds_lpd[-200*12:] -xr_quadtrend(SST_gm_ds_lpd[-200*12:] )+1.1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi[-200*12:] -xr_quadtrend(SST_xm_ds_lpi[-200*12:] ),
            SST_gm_ds_lpi[-200*12:] -xr_quadtrend(SST_gm_ds_lpi[-200*12:] )-.6, alpha=.1)
plt.scatter(SST_xm_ds_had[-100*12:] -xr_quadtrend(SST_xm_ds_had[-100*12:] ),
            [-1.1]*(100*12), alpha=.1)
plt.ylabel('global mean SST' , fontsize=14)
plt.xlabel('60S-60N mean SST', fontsize=14)
plt.tight_layout()

In [ ]:
# lowpass deseasonalizing
f, ax = plt.subplots(1, 2, figsize=(12,5))
for i in range(2):
    ax[i].tick_params(labelsize=14)

ax[0].plot(SST_xm_ctrl.time[:120]/12   , SST_xm_ctrl   [:120])
ax[0].plot(SST_xm_ctrl.time[:120]/12   , SST_xm_ds_ctrl[:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_rcp    [:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_ds_rcp [:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_ds_rcp [:120])

ax[1].plot(SST_xm_ctrl.time/12    , SST_xm_ctrl   )
ax[1].plot(SST_xm_ctrl.time/12    , SST_xm_ds_ctrl)
ax[1].plot(SST_xm_rcp .time/12+220, SST_xm_rcp    )
ax[1].plot(SST_xm_rcp .time/12+220, SST_xm_ds_rcp )
ax[1].plot(SST_xm_had .time/365   , SST_xm_had    )
ax[1].plot(SST_xm_had .time/365   , SST_xm_ds_had )
plt.tight_layout()
plt.savefig(f'{path_results}/SST/SST_deseasonalizing_global_mean')

In [ ]:
plt.figure(figsize=(12,5))
plt.tick_params(labelsize=14)

plt.plot(SST_gm_ctrl.time/12 +1950, SST_gm_ctrl   , c='C0', lw=.3, alpha=.5)
plt.plot(SST_gm_rcp .time/12 +2200, SST_gm_rcp    , c='C1', lw=.3, alpha=.5)
plt.plot(SST_gm_lpd .time/365+1350, SST_gm_lpd    , c='C2', lw=.3, alpha=.5)
plt.plot(SST_gm_lpi .time/365-1600, SST_gm_lpi    , c='C3', lw=.3, alpha=.5)
plt.plot(SST_gm_ctrl.time/12 +1950, SST_gm_ds_ctrl, c='C0')
plt.plot(SST_gm_rcp .time/12 +2200, SST_gm_ds_rcp , c='C1')
plt.plot(SST_gm_lpd .time/365+1350, SST_gm_ds_lpd , c='C2')
plt.plot(SST_gm_lpi .time/365-1600, SST_gm_ds_lpi , c='C3')

plt.plot(SST_xm_ctrl.time/12 +1950, SST_xm_ctrl   , c='C0', lw=.3, alpha=.5)
plt.plot(SST_xm_rcp .time/12 +2200, SST_xm_rcp    , c='C1', lw=.3, alpha=.5)
plt.plot(SST_xm_lpd .time/365+1350, SST_xm_lpd    , c='C2', lw=.3, alpha=.5)
plt.plot(SST_xm_lpi .time/365-1600, SST_xm_lpi    , c='C3', lw=.3, alpha=.5)
plt.plot(SST_xm_had .time/365+2350, SST_xm_had    , c='C4', lw=.3, alpha=.5)
plt.plot(SST_xm_ctrl.time/12 +1950, SST_xm_ds_ctrl, c='C0')
plt.plot(SST_xm_rcp .time/12 +2200, SST_xm_ds_rcp , c='C1')
plt.plot(SST_xm_lpd .time/365+1350, SST_xm_ds_lpd , c='C2')
plt.plot(SST_xm_lpi .time/365-1600, SST_xm_ds_lpi , c='C3')
plt.plot(SST_xm_had .time/365+2350, SST_xm_ds_had , c='C4')

plt.ylabel('global / 60S-60N mean SST [$^\circ$C]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)

plt.tight_layout()